<a href="https://colab.research.google.com/github/m-bashari-m/vehicle-color-recognition/blob/main/src/xyz_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py

--2022-06-01 21:10:16--  https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8912 (8.7K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]   8.70K  --.-KB/s    in 0s      

2022-06-01 21:10:16 (41.0 MB/s) - ‘utils.py’ saved [8912/8912]



In [2]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [3]:
import pandas as pd
import numpy as np
import os

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

!pip install -q tensorflow-io
import tensorflow_io as tfio

from utils import ModelCreator, get_train_val_ds, ErrorAnalyzer, get_class_weight

     |████████████████████████████████| 25.9 MB 66.4 MB/s 


In [4]:
BATCH_SIZE = 32
IMG_SIZE = (512, 512)
AUTOTUNE = tf.data.AUTOTUNE
N_CLASSES = 16
HUB_URL = 'https://tfhub.dev/google/bit/m-r50x1/1'

In [5]:
dataset_dir = os.path.join('drive', 'MyDrive', 'cars')
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')

In [6]:
train_ds, val_ds = get_train_val_ds(train_dir, val_dir, batch_size=BATCH_SIZE)

Found 16580 files belonging to 16 classes.
Found 3508 files belonging to 16 classes.


In [7]:
classes, class_weight = get_class_weight()

In [8]:
train_ds = (
    train_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    map(lambda img, lbl: (tfio.experimental.color.rgb_to_xyz(img), lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

val_ds = (
    val_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    map(lambda img, lbl: (tfio.experimental.color.rgb_to_xyz(img), lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

In [9]:
xyz_creator = ModelCreator(HUB_URL, 'xyz-model')
xyz_model = xyz_creator.make_model()
callbacks = xyz_creator.get_callbacks()

Model: "xyz-model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 2048)              23500352  
                                                                 
 dense (Dense)               (None, 600)               1229400   
                                                                 
 dropout (Dropout)           (None, 600)               0         
                                                                 
 dense_1 (Dense)             (None, 16)                9616      
                                                                 
Total params: 24,739,368
Trainable params: 1,239,016
Non-trainable params: 23,500,352
_________________________________________________________________


### First day of training

In [13]:
history = xyz_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=10,
                        class_weight=class_weight)

Epoch 1/10
519/519 [==============================] - 639s 1s/step - loss: 1124.2683 - precision: 0.5762 - recall: 0.5759 - auc: 0.4053 - accuracy: 0.5761
Epoch 2/10
519/519 [==============================] - 625s 1s/step - loss: 1246.8962 - precision: 0.6368 - recall: 0.6368 - auc: 0.4664 - accuracy: 0.6368
Epoch 3/10
519/519 [==============================] - 657s 1s/step - loss: 739.6858 - precision: 0.6897 - recall: 0.6897 - auc: 0.5347 - accuracy: 0.6897
Epoch 4/10
519/519 [==============================] - 685s 1s/step - loss: 541.9413 - precision: 0.7149 - recall: 0.7149 - auc: 0.5711 - accuracy: 0.7149
Epoch 5/10
519/519 [==============================] - 650s 1s/step - loss: 433.9911 - precision: 0.7261 - recall: 0.7260 - auc: 0.5889 - accuracy: 0.7261
Epoch 6/10
519/519 [==============================] - 637s 1s/step - loss: 306.7163 - precision: 0.7503 - recall: 0.7501 - auc: 0.6285 - accuracy: 0.7502
Epoch 7/10
519/519 [==============================] - 638s 1s/step - loss:

In [14]:
xyz_model.save_weights('drive/MyDrive/checkpoints/xyz-weights-10.h5')

### Second day

In [13]:
xyz_model.load_weights('drive/MyDrive/checkpoints/xyz-weights-10.h5')

In [ ]:
analyzer = ErrorAnalyzer(xyz_model, val_ds, classes, 'XYZ-model')

Making confusion matrix:


  0%|          | 0/55 [00:00<?, ?it/s]

In [ ]:
analyzer.plot_confusion_mat()
analzyer.evaluaet_model()